# Mount_virtual-media_iso

<div class="alert alert-block alert-info">
<i>Hello there.!<br></i> Welcome to <b>python-iLOrest-library</b> <br> Here is an example to <b>mount_virtual_media_iso</b> <br> As you are here let's get it done easily.<br>
    To perform this u need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style = "none">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

<div class="alert alert-block alert-success">
    <h1> 1.</h1>
Below we are importing all the necessary modules from the ilorest library!.<br>
    <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys </a>
    </li>
    <li>json </li>
    <li> redfish </li>
    <li>get_resource_directory</li>
</div>

In [1]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError
from get_resource_directory import get_resource_directory

ModuleNotFoundError: No module named 'get_resource_directory'

<div class = "alert alert-block alert-success">
    <h1> 2. </h1>This is our mount_virtual-media_iso function overview.<br>
    <i>Let me give you a walk through of it!</i>
    <br>
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present.<br>

1. In the <b>change_bios_settings</b> function we are first using <b>resource_instances</b> variable to obtain information about the resource directory by using get_resource_directory module.
<br>
    
2. Retrieving information on the server's management interfaces using the Redfish API client, including members and their properties and retrieving URI for virtual media using Redfish API client.
    <br>

3. Else Using the Redfish API client and the Resource Directory, iterating through resource instances, and obtaining the URI for the virtual media collection.
    <br>
4. If <b>virtual_media_uri </b> is not <b>None</b> then If a slot matches, the program fetches the URI for adding media into that slot and then sends a <b>POST</b> request to the supplied URL to insert the ISO image. Additionally, the code sends a <b>PATCH </b>request to set the option if a parameter is provided for the server to restart from the virtual media.
    <br>

5. Handling the error after inserting the iso image in slot 
    If status code is <b>400</b> the code attempts to retrieve and print extended error message .
    If status code is <b> 200 OK</b> then it will print error meassage and status code.
    Else the insertion is <b>success</b> then code print result in <b>JSON</b> format.

 </div>

In [2]:
def mount_virtual_media_iso(_redfishobj, iso_url, media_type, boot_on_next_server_reset):
 #Initializing variables for virtual media API responses.
    virtual_media_uri = None
    virtual_media_response = []
#Retrieving resource directory instances using Redfish API client.
    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
        virtual_media_uri = managers_members_response.obj['VirtualMedia']['@odata.id']
    else:
        for instance in resource_instances:
            #Use Resource directory to find the relevant URI
            if '#VirtualMediaCollection.' in instance['@odata.type']:
                virtual_media_uri = instance['@odata.id']

    if virtual_media_uri:
        #retriving information Virtual media using redfish object and specified parameter
        virtual_media_response = _redfishobj.get(virtual_media_uri)
        #iterates through each virtual media to find proper slot
        for virtual_media_slot in virtual_media_response.obj['Members']:
            data = _redfishobj.get(virtual_media_slot['@odata.id'])
            if media_type in data.dict['MediaTypes']:
                virtual_media_mount_uri = data.obj['Actions']['#VirtualMedia.InsertMedia']['target']
                post_body = {"Image": iso_url}

                if iso_url:
                    resp = _redfishobj.post(virtual_media_mount_uri, post_body)
                    if boot_on_next_server_reset is not None:
                        patch_body = {}
                        patch_body["Oem"] = {"Hpe": {"BootOnNextServerReset": \
                                                 boot_on_next_server_reset}}
                        boot_resp = _redfishobj.patch(data.obj['@odata.id'], patch_body)
                        if not boot_resp.status == 200:
                            sys.stderr.write("Failure setting BootOnNextServerReset")
                #handling error after inserting iso image in slot 
                    if resp.status == 400:
                        try:
                            print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                                                                    sort_keys=True))
                        except Exception as excp:
                            sys.stderr.write("A response error occurred, unable to access iLO"
                                             "Extended Message Info...")
                    elif resp.status != 200:
                        sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
                    else:
                        print("Success!\n")
                        print(json.dumps(resp.dict, indent=4, sort_keys=True))
                break
        


<div class = "alert alert-block alert-success">
    <h1> 3. </h1>This is Main function overview.<br>
    <i>Let me give you a walk through of it!</i>
    <br>
    <br>
1. If server is runnig in locally then url,login_account,Login-password will be <b> None</b>.
      Else then we need to enter the proper credential.
    <br>
<br>
    
2. <b>media_url</b> will provide access and mount the iso image in server side and we initial <b>media_type</b> to <b> CD</b>.<br>
    <br>

3. Utilising the Redfish API to create a Redfish client object with the provided login information and log in to the server.<br>
    <br>

4. resolving the exception that is thrown when the server cannot be reached or does not implement the Redfish API, printing a message of error to standard error output, and terminating the program<br>
    <br>
5. Using the Redfish API client object and the given arguments to mount a virtual media ISO image on a server.<br>
    

6. REDFISHOBJ.logout() will take care of logging you out of the ilo.

 </div>

In [1]:
if __name__ == "__main__":
    # When running on the server locally use the following commented values
    #SYSTEM_URL = None
    #LOGIN_ACCOUNT = None
    #LOGIN_PASSWORD = None

    # When running remotely connect using the secured (https://) address,
    # account name, and password to send https requests
    # SYSTEM_URL acceptable examples:
    # "https://10.0.0.100"
    # "https://ilo.hostname"
    SYSTEM_URL = "https://10.0.0.100"
    LOGIN_ACCOUNT = "admin"
    LOGIN_PASSWORD = "password"
 # used to access and mount iso image on server side
    MEDIA_URL = "http://<SERVER_URL>/media.iso"

    #specify the type of content the media represents
    MEDIA_TYPE = "CD" #current possible options: Floppy, USBStick, CD, DVD
    #specify if the server should attempt to boot this media on system restart
    BOOT_ON_NEXT_SERVER_RESET = True

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    mount_virtual_media_iso(REDFISHOBJ, MEDIA_URL, MEDIA_TYPE, BOOT_ON_NEXT_SERVER_RESET)
    REDFISHOBJ.logout()


Enter System URL: 120.1.114.51
Enter Name: tann
Enter password: none
Enter bios password: noen
Enter bios property you wish to change: yes
Enter Value of the property to change: none


NameError: name 'ServerDownOrUnreachableError' is not defined

<div class="alert alert-block alert-warning">
<b> When running on the server locally : </b>  use the following values <br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address,account name, and password to send https requests <br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

<div class = "alert alert-block alert-success">In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>